In [1]:
import time
import os
from dotenv import load_dotenv

from openai import AzureOpenAI

# gets API Key from environment variable OPENAI_API_KEY
load_dotenv()
api_endpoint = os.getenv("OPENAI_URI")
api_key = os.getenv("OPENAI_KEY")
api_version = os.getenv("OPENAI_VERSION")
api_deployment_name = os.getenv("OPENAI_GPT_DEPLOYMENT")


client = AzureOpenAI(api_key=api_key,
api_version=api_version,
azure_endpoint=api_endpoint)


In [2]:
assistant = None
print(api_deployment_name)

try:
    assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model=api_deployment_name,
)
except Exception as e:
    print(e)

gpt-4


In [3]:
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)

In [4]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Jane Doe. The user has a premium account.",
)

In [10]:
print("checking assistant status. ")

while True:
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    if run.status == "completed":
        print("Assistant has completed the task.")
        messages = client.beta.threads.messages.list(thread_id=thread.id)

        for message in messages:
            print(f"{message.role}: {message.content}")
        
        break
    
    else:
        print("Assistant is still working. Please wait.")
        time.sleep(5)

checking assistant status. 
Assistant has completed the task.
assistant: [MessageContentText(text=Text(annotations=[], value='The solution to the equation `3x + 11 = 14` is `x = 1.0`, Jane Doe.'), type='text')]
assistant: [MessageContentText(text=Text(annotations=[], value="Sure, Jane Doe. The solution can be found by a straightforward manipulation of the equation. Here are the specific steps:\n\n1. We subtract 11 from both sides of the equation which gives us `3x = 14 - 11`.\n2. We then divide both sides of the resulting equation by 3, which gives us `x = (14 - 11) / 3`.\n\nLet's calculate that now."), type='text')]
user: [MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')]
